In [14]:
import sys
import os
import numpy as np
import cv2
import scipy.sparse as sp
import torch
from matplotlib import pyplot as plt

HybridGNet_pth = os.path.abspath('../CheXmask-Database-main/HybridGNet')
if HybridGNet_pth not in sys.path: sys.path.append(HybridGNet_pth)

from models.HybridGNet2IGSC import Hybrid
from utils.utils import scipy_to_torch_sparse, genMatrixesLungsHeart

In [15]:
import torch_geometric
print(torch_geometric.__version__)

2.4.0


In [16]:
def getDenseMask(RL, LL, H):
    img = np.zeros([1024,1024], dtype = 'uint8')
    
    RL = RL.reshape(-1, 1, 2).astype('int')
    LL = LL.reshape(-1, 1, 2).astype('int')
    H = H.reshape(-1, 1, 2).astype('int')

    img = cv2.drawContours(img, [RL], -1, 1, -1)
    img = cv2.drawContours(img, [LL], -1, 1, -1)
    img = cv2.drawContours(img, [H], -1, 2, -1)
    
    return img

In [17]:
device = "cpu"


A, AD, D, U = genMatrixesLungsHeart()
N1 = A.shape[0]
N2 = AD.shape[0]

A = sp.csc_matrix(A).tocoo()
AD = sp.csc_matrix(AD).tocoo()
D = sp.csc_matrix(D).tocoo()
U = sp.csc_matrix(U).tocoo()

D_ = [D.copy()]
U_ = [U.copy()]

config = {}

config['n_nodes'] = [N1, N1, N1, N2, N2, N2]
A_ = [A.copy(), A.copy(), A.copy(), AD.copy(), AD.copy(), AD.copy()]
A_t, D_t, U_t = ([scipy_to_torch_sparse(x).to(device) for x in X] for X in (A_, D_, U_))

config['latents'] = 64
config['inputsize'] = 1024

f = 32
config['filters'] = [2, f, f, f, f//2, f//2, f//2]
config['skip_features'] = f


hybrid = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid.load_state_dict(torch.load(os.path.join(HybridGNet_pth, "../Weights/SegmentationModel/bestMSE.pt"), 
                                  map_location=torch.device('cpu')))
hybrid.eval()
print('Model loaded')


Model loaded


/var/folders/ny/r0bq7b513_qd07rknvttg91r0000gn/T/ipykernel_7868/2815268437.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hybrid.load_state_dict(torch.load(os.path.joi

In [20]:
img = cv2.imread('../data/mimic-cxr-demo/test/s50010747.jpg', 0) / 255.0
img = cv2.resize(img, (1024, 1024))
# plt.imshow(img, cmap='gray')

data = torch.from_numpy(img).unsqueeze(0).unsqueeze(0).to(device).float()
# print(torch.min(data))
# print(torch.max(data))
# print(data.shape)

with torch.no_grad():
    output = hybrid(data)

In [21]:
output

(tensor([[[0.3804, 0.1670],
          [0.3512, 0.1708],
          [0.3245, 0.1900],
          [0.2975, 0.2144],
          [0.2741, 0.2416],
          [0.2555, 0.2726],
          [0.2393, 0.3043],
          [0.2262, 0.3321],
          [0.2132, 0.3621],
          [0.2017, 0.3936],
          [0.1919, 0.4257],
          [0.1819, 0.4588],
          [0.1747, 0.4898],
          [0.1671, 0.5217],
          [0.1573, 0.5516],
          [0.1511, 0.5825],
          [0.1466, 0.6121],
          [0.1372, 0.6443],
          [0.1277, 0.6781],
          [0.1190, 0.7095],
          [0.1110, 0.7437],
          [0.1125, 0.7752],
          [0.1362, 0.7487],
          [0.1683, 0.7241],
          [0.2036, 0.7076],
          [0.2387, 0.6989],
          [0.2711, 0.7016],
          [0.3019, 0.7109],
          [0.3316, 0.7233],
          [0.3553, 0.7345],
          [0.3585, 0.7019],
          [0.3637, 0.6605],
          [0.3733, 0.6215],
          [0.3883, 0.5819],
          [0.3976, 0.5408],
          [0.4043, 0